In [2]:
import numpy as np

import pandas as pd



In [22]:
df = pd.DataFrame({"key1" : ["a", "a", None, "b", "b", "a", None],
    "key2" : pd.Series([1, 2, 1, 2, 1, None, 1],
    dtype="Int64"),
    "data1" : np.random.standard_normal(7),
    "data2" : np.random.standard_normal(7)})

# df.groupby(df["key1"]).aggregate([np.mean, min, max]).stack(level=1).reset_index().set_index("key1")

df.groupby("key1").aggregate([np.mean, min, max]).stack(level=1).reset_index().set_index("key1")


C:\Users\paulb\AppData\Local\Temp\ipykernel_16756\2747561318.py:9: FutureWarning: The provided callable <function mean at 0x000001F120E862A0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df.groupby("key1").aggregate([np.mean, min, max]).stack(level=1).reset_index().set_index("key1")
C:\Users\paulb\AppData\Local\Temp\ipykernel_16756\2747561318.py:9: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  df.groupby("key1").aggregate([np.mean, min, max]).stack(level=1).reset_index().set_index("key1")
C:\Users\paulb\AppData\Local\Temp\ipykernel_16756\2747561318.py:9: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of p

,level_1,key2,data1,data2
key1,,,,
a,mean,1.5,0.680940,-0.445635
a,min,1.0,0.235458,-0.720560
a,max,2.0,0.991232,-0.190709
b,mean,1.5,-0.001092,-0.873546
b,min,1.0,-0.206897,-1.433110
b,max,2.0,0.204714,-0.313982


In [ ]:
rn_gen = np.random.default_rng(1)

people = pd.DataFrame(rn_gen.normal(1, 0.1, 25).reshape(5, 5),
    columns=["a", "b", "c", "d", "e"],
    index=["Joe", "Steve", "Wanda", "Jill", "Trey"])

people.groupby(lambda x: x.endswith("e")).sum()

col_classifier = dict()
for letter in "a", "e", "i", "o", "u": 
    col_classifier.update({letter: "vowel"})
for letter in "b", "c", "d", "f", "g", "h": 
    col_classifier.update({letter: "const"})

# we can use dictionaries directly to classify groups (rather than spawning a separate column with the mapping)
people.T.groupby(col_classifier).sum().T # note we use .T method, although using axis = "columns" was recommended in book but appears to be depracated for T

,const,vowel
Joe,2.984890,2.125094
Steve,3.040874,2.074051
Wanda,2.964735,1.954630
Jill,2.896536,2.034165
Trey,3.202519,1.729698


In [170]:
# Set a random seed for reproducibility
np.random.seed(42)

# Define possible categories
categories = ['A', 'B', 'C']

# Generate the dataset
data = {
    'ID': range(1, 11),
    'Category': np.random.choice(categories, size=10),
    'Value1': np.random.randint(10, 100, size=10),
    'Value2': np.random.normal(loc=50, scale=10, size=10).round(2)
}

df = pd.DataFrame(data)

df2 = df

In [115]:
df.groupby("Category")["Value1"].agg([("average", "mean"), ("maximum", "max")])

df.groupby("Category").agg({"Value2": ["min", "max"], "Value1": "min"})

df.groupby("Category").agg(
    value2min = pd.NamedAgg(column="Value2", aggfunc="min"),
    value1min = pd.NamedAgg(column="Value1", aggfunc="min"), 
    value1max = pd.NamedAgg(column="Value1", aggfunc="max")
)

,value2min,value1min,value1max
Category,,,
A,23.87,12,31
B,34.76,11,11
C,40.76,33,97


In [ ]:
df_groups = pd.cut(df.Value1, 3)
df.groupby(df_groups)["Value2"].agg(["mean", "max", "min", "count"]).reset_index()

df.loc[[1, 8], "Value2"] = np.nan # create some NA cells

def missing_to_mean(s):
    return s.fillna(s.mean())

v2 = df.groupby(df_groups)["Value2"].apply(missing_to_mean)
v2.index = v2.index.get_level_values(1)


df["Value2_with_missing"] = v2
df

v3 = df.groupby(df_groups, group_keys=False)["Value2"].apply(missing_to_mean)
v3 - v2

C:\Users\paulb\AppData\Local\Temp\ipykernel_16756\3587759283.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(df_groups)["Value2"].agg(["mean", "max", "min", "count"]).reset_index()
C:\Users\paulb\AppData\Local\Temp\ipykernel_16756\3587759283.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  v2 = df.groupby(df_groups)["Value2"].apply(missing_to_mean)
C:\Users\paulb\AppData\Local\Temp\ipykernel_16756\3587759283.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=Tr

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
5    0.0
6    0.0
7    0.0
8    0.0
9    0.0
Name: Value2, dtype: float64

In [ ]:
# we can use .transform() insteaad of .apply() to retain the original structure...

# this is similar to when using mutate() with group_by() in R

df2["V2_diff_from_mean"] = df2.groupby("Category")["Value2"].transform("mean") - df2["Value2"]

df2.sort_values(by = ["Category", "Value2"])

,ID,Category,Value1,Value2,V2_diff_from_mean
4,5,A,12,23.87,18.836667
1,2,A,84,44.75,-2.043333
5,6,A,31,59.50,-16.793333
7,8,B,11,34.76,0.000000
3,4,C,33,40.76,5.190000
9,10,C,39,42.58,3.370000
0,1,C,84,44.19,1.760000
2,3,C,97,44.29,1.660000
8,9,C,97,45.72,0.230000
6,7,C,62,58.16,-12.210000
